In [1]:
import pandas as pd
from hs_gimme.db_facade.db_facade_factory import get_mongo_client_db
from hs_gimme.logging_service.problems_only_logging_service import ProblemsOnlyLoggingService
from hs_gimme.db_facade.db_facade_factory import get_mongo_client_db
from hs_recommendations_api_client import get_recommendations_api_client
from tqdm import tqdm
from json2html import *
from hs_gimme.s3.resume_storage import ResumeStorage
from hs_gimme.hs_object_functions.application import get_selected_resume
from hs_gimme.constants.objects.application import ResumeData
import logging
import pandas as pd
import itertools
from IPython.core.display import display, HTML

/var/folders/_p/8_m91crj199_rd7sndkxncc80000gp/T/ipykernel_68531/969554964.py:14: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
logger = logging.getLogger()
resume_storage = ResumeStorage(logger)

In [3]:
path = '~/Downloads/fetch_metrics_raw.csv'

In [4]:
env = 'production'

In [5]:
df = pd.read_csv(path)

In [6]:
df.head()

,Unnamed: 0,account_id,req_id,current_talent_id,filter,group_name,predicted,order
0,0,alabama,ABV1905074,4cbac0c8-9c27-45e8-abcb-909236ed86fb,ALL,Negative Rejected,0,NaN
1,1,alabama,ABV1905074,bdb2baf8-3564-4258-bcb1-b5569d31e0c0,ALL,Negative Rejected,1,1.0
2,2,alabama,ABV1905074,552e7aa3-6ac5-4a50-a12a-743bf5f54228,ALL,Positive Offer+ With Experience,1,2.0
3,3,alabama,ABV1905074,09280f9c-e9d7-4670-8f26-c0ae1a38edb4,ALL,Positive Offer+ With Experience,1,0.0
4,4,alabama,ABV1905074,2ea57388-afb7-41aa-b164-9578e8b7cb6c,ALL,Random from other reqs,0,NaN


In [7]:
df['account_id'].value_counts()

moscow      20646
jamaica     11313
indiana      6825
dallas       6018
alabama      5949
oxford       4800
tampa        2655
sofia        2592
siena        1335
utah         1287
essex         939
awasa         690
colorado      666
Name: account_id, dtype: int64

In [8]:
df = df[df['account_id'].isin({'indiana', 'moscow', 'jamaica'})]
len(df)
        

38784

In [39]:
reqs = {}

reqs_p = [ 'english_job_title', 'job_education', 
           'management_level', 'min_years_of_relevant_experience', 
           'seniority_level', 'top_category',  'sub_category', 'job_title', 'job_listing']
num_of_accounts = len(df['account_id'].unique())
for i, (account, account_df) in enumerate(df.groupby('account_id')):
    mongo = get_mongo_client_db(env, account)
    
    req_ids = list(account_df['req_id'].unique())
    print(f'{i}/{num_of_accounts} {account} {len(req_ids)} reqs')
    
    for r in mongo.req.find({'_id': {'$in': req_ids}}, reqs_p):
        reqs[r['_id']] = r

0/2 indiana 1487 reqs
1/2 moscow 3451 reqs
2023-06-01T10:18:07.235342Z [warning  ] SlowLog                        command_name=getMore connection_id=mongodb-3c.prod.omcomcom.com:27029 duration_ms__numeric=13571.931 duration_threshold__numeric=10000 environment=local file_path=/Users/dima/hiredscore/gimme/hs_gimme/db_facade/slow_logs_utils.py function_name=write_slow_log hostname=Dima-Shulga-MacBook-Pro line_number=11 module=hs_gimme.db_facade.slow_logs_utils pid=68531 request_id=1527729602 source=PyMongo stack_snippet=
  File "/var/folders/_p/8_m91crj199_rd7sndkxncc80000gp/T/ipykernel_68531/2334949316.py", line 13, in <module>
    for r in mongo.req.find({'_id': {'$in': req_ids}}, reqs_p):

2023-06-01T10:18:17.340198Z [warning  ] SlowLog                        command_name=getMore connection_id=mongodb-3c.prod.omcomcom.com:27029 duration_ms__numeric=10098.552 duration_threshold__numeric=10000 environment=local file_path=/Users/dima/hiredscore/gimme/hs_gimme/db_facade/slow_logs_utils.py

In [38]:
for f in [ 'english_job_title', 'job_education', 'management_level', 'min_years_of_relevant_experience', 'seniority_level',
           'top_category',  'sub_category'
        ]:
    df['req_' + f] = df['req_id'].apply(lambda r: reqs[r].get(f))

In [11]:
def get_exp(x):
    # if x < 1:
    #     return '0'
    if x < 3:
        return '<3'
    # if x <= 5:
    #     return '3-5'
    
    if x >= 3:
        return '>=3'

df['exp'] = df['req_min_years_of_relevant_experience'].apply(get_exp)

In [12]:
df['exp'].value_counts()

>=3    11163
<3     11016
Name: exp, dtype: int64

In [13]:
df['req_seniority_level'].value_counts()

junior         22497
entry_level     7587
senior          6576
manager          957
director         786
intern           183
student          171
vp                21
c-level            6
Name: req_seniority_level, dtype: int64

In [14]:
df = df[df['account_id'].isin({'indiana', 'moscow'})]
df['account_id'].value_counts()

moscow     20646
indiana     6825
Name: account_id, dtype: int64

In [15]:
df = df[df['req_seniority_level'].isin({'senior', 'junior'})]
df.groupby(['account_id', 'req_seniority_level']).size()

account_id  req_seniority_level
indiana     junior                  3645
            senior                  2262
moscow      junior                 13536
            senior                   687
dtype: int64

In [16]:
def sample_seniority(x):
    ss = 100
    if x['req_seniority_level'].unique()[0] == 'junior':
        ss = 40
        
    return x.sample(ss)
    
    
sample = df.groupby(['account_id', 'req_seniority_level']).apply(sample_seniority).reset_index(drop=True)

In [17]:
sample.groupby(['account_id', 'req_seniority_level']).size()

account_id  req_seniority_level
indiana     junior                  40
            senior                 100
moscow      junior                  40
            senior                 100
dtype: int64

In [18]:
sample.head()

,Unnamed: 0,account_id,req_id,current_talent_id,filter,group_name,predicted,order,req_english_job_title,req_job_education,req_management_level,req_min_years_of_relevant_experience,req_seniority_level,req_top_category,req_sub_category,exp
0,10855,indiana,INTJR0186470,686d90ef-c8fb-4122-8da9-2f39279ae17c,ALL,Positive Offer+ With Experience,1,1.0,Cloud Software Engineer,masters,Experienced,4.0,junior,IT_&_Software_Development,Software_=_Web_Development ~ Embedded,>=3
1,11176,indiana,INTJR0181976,d18f4cda-19fc-4b2b-8eee-2d4735d9b52e,ALL,Positive Offer+ With Experience,1,3.0,oneAPI Development Software Architect (Remote),bachelors,Experienced,2.0,junior,IT_&_Software_Development,Software_=_Web_Development,<3
2,14367,indiana,INTJR0196855,3745f304-e446-4104-b25b-b4969b539f8b,ALL,Random from other reqs,1,0.0,Analog Engineer,masters,Entry Level,1.0,junior,Engineering,Electrical_=_Electronic_Engineering ~ Hardware...,<3
3,9075,indiana,INTJR0117739,8aabcc5a-f9b5-4a4a-9748-ae273e1744ca,ALL,Positive Offer+ With Experience,0,NaN,Quantum Device Measurement Engineer,doctorate,Experienced,4.0,junior,Engineering,Electrical_=_Electronic_Engineering,>=3
4,9665,indiana,INTJR0185482,9f3ad4ee-f383-45c1-9002-db9387bc8fc8,ALL,Random from other reqs,0,NaN,Graphics Software Engineer,masters,Experienced,3.0,junior,IT_&_Software_Development,Software_=_Web_Development ~ Embedded,>=3


In [30]:
talents = {}

talents_p = ['current_talent_id', 
             'attachments', 
             'selected_resume_hash', 
             'new_attachments', 
             'new_selected_resume_hash'] 
num_of_accounts = len(sample['account_id'].unique())
for i, (account, account_df) in enumerate(sample.groupby('account_id')):
    mongo = get_mongo_client_db(env, account)

        
    talent_ids = list(account_df['current_talent_id'].unique())
    print(f'{i}/{num_of_accounts} {account} {len(talent_ids)} talents')
    
    for r in mongo.application.find({'current_talent_id': {'$in': talent_ids}}, talents_p):
        talents[r['current_talent_id']] = r

0/2 indiana 140 talents
2023-06-01T08:48:51.528726Z [warning  ] SlowLog                        command_name=getMore connection_id=mongodb-3c.prod.omcomcom.com:27029 duration_ms__numeric=41802.955 duration_threshold__numeric=10000 environment=local file_path=/Users/dima/hiredscore/gimme/hs_gimme/db_facade/slow_logs_utils.py function_name=write_slow_log hostname=Dima-Shulga-MacBook-Pro line_number=11 module=hs_gimme.db_facade.slow_logs_utils pid=68531 request_id=629306008 source=PyMongo stack_snippet=
  File "/var/folders/_p/8_m91crj199_rd7sndkxncc80000gp/T/ipykernel_68531/2155183365.py", line 16, in <module>
    for r in mongo.application.find({'current_talent_id': {'$in': talent_ids}}, talents_p):

1/2 moscow 140 talents
2023-06-01T08:49:08.999756Z [warning  ] SlowLog                        command_name=getMore connection_id=mongodb-3c.prod.omcomcom.com:27029 duration_ms__numeric=15728.695 duration_threshold__numeric=10000 environment=local file_path=/Users/dima/hiredscore/gimme/hs_gim

In [31]:
def add_html_paragraph(text):
    return '<p>' + text.replace('\n', '<br>') + '</p>'

def load_resume_bytes(app_from_mongo: str,  _resume_storage: ResumeStorage):
    resume_data = get_selected_resume(app_from_mongo)
    unique_key = resume_data.get(ResumeData.PDF_UNIQUE_KEY) or resume_data[ResumeData.UNIQUE_KEY]
    resume_bytes = _resume_storage.get_resume_bytes(resume_data, unique_key=unique_key)
    return resume_bytes

In [34]:
for talent_id in sample['current_talent_id'].unique():
    try:
        candidate = talents[talent_id]
        resume_bytes = load_resume_bytes(candidate, resume_storage)

        resume_path = f'/Users/dima/Downloads/for_mer/fetch_2/{talent_id}.pdf'
        with open(resume_path, 'wb') as f:
            f.write(resume_bytes)
    except:
        pass

Getting resume file for (bucket=hs-production-resumes, unique_key=indiana/b3d010ff4d7b838f8305c2c729d4dc00/CV_KarenGarcia.pdf)
Getting resume file for (bucket=hs-production-resumes, unique_key=indiana/f2574483fa08eeae79e400ff40c41974/EttoreTiottoCV_v3.pdf)
Getting resume file for (bucket=hs-production-resumes, unique_key=indiana/b5dfe8de0f769d58bf0c8135d8c84c83/Alexander_Samochin.pdf)
Getting resume file for (bucket=hs-production-resumes, unique_key=indiana/25d2132cf35cb0fcc6f7e389c4834cda/Resume_DK.pdf)
Getting resume file for (bucket=hs-production-resumes, unique_key=indiana/058403ca5464a38cb9389623d0f3b1a0/גיליון ציונים סמסטר 7 (2).pdf)
Getting resume file for (bucket=hs-production-resumes, unique_key=indiana/1593d2c33bb06a2c80ce6158a6dc7131/YashwanthK_CV.pdf)
Getting resume file for (bucket=hs-production-resumes, unique_key=indiana/7fe79f74e75b80a3797aac4bd673554f/CV_Fernando_FG.pdf)
Getting resume file for (bucket=hs-production-resumes, unique_key=indiana/aa9434cd3c777e0fb8ee968de

In [40]:
for req_id in sample['req_id'].unique():  
    
    req = reqs[req_id]
    req_html = "<h1>" + req['job_title'] + "</h3"
    for k, t in req['job_listing'].items():
        req_html += '<h3>' + k + '</h3>'
        req_html += add_html_paragraph(t)
        
    req_path = f'/Users/dima/Downloads/for_mer/fetch_2/{req_id}.htm'
    with open(req_path, 'w') as f:
        f.write(req_html)
        print(req_path)

/Users/dima/Downloads/for_mer/fetch_2/INTJR0186470.htm
/Users/dima/Downloads/for_mer/fetch_2/INTJR0181976.htm
/Users/dima/Downloads/for_mer/fetch_2/INTJR0196855.htm
/Users/dima/Downloads/for_mer/fetch_2/INTJR0117739.htm
/Users/dima/Downloads/for_mer/fetch_2/INTJR0185482.htm
/Users/dima/Downloads/for_mer/fetch_2/INTJR0174303.htm
/Users/dima/Downloads/for_mer/fetch_2/INTJR0166120.htm
/Users/dima/Downloads/for_mer/fetch_2/INTJR0181433.htm
/Users/dima/Downloads/for_mer/fetch_2/INTJR0175524.htm
/Users/dima/Downloads/for_mer/fetch_2/INTJR0153632.htm
/Users/dima/Downloads/for_mer/fetch_2/INTJR0150504.htm
/Users/dima/Downloads/for_mer/fetch_2/INTJR0185797.htm
/Users/dima/Downloads/for_mer/fetch_2/INTJR0132632.htm
/Users/dima/Downloads/for_mer/fetch_2/INTJR0188171.htm
/Users/dima/Downloads/for_mer/fetch_2/INTJR0146757.htm
/Users/dima/Downloads/for_mer/fetch_2/INTJR0198775.htm
/Users/dima/Downloads/for_mer/fetch_2/INTJR0195477.htm
/Users/dima/Downloads/for_mer/fetch_2/INTJR0135288.htm
/Users/dim

In [43]:
sample.to_csv('/Users/dima/Downloads/for_mer/fetch_2/indian_moscow_junior_senior.csv', index=False)

In [42]:
sample

,Unnamed: 0,account_id,req_id,current_talent_id,filter,group_name,predicted,order,req_english_job_title,req_job_education,req_management_level,req_min_years_of_relevant_experience,req_seniority_level,req_top_category,req_sub_category,exp
0,10855,indiana,INTJR0186470,686d90ef-c8fb-4122-8da9-2f39279ae17c,ALL,Positive Offer+ With Experience,1,1.0,Cloud Software Engineer,masters,Experienced,4.0,junior,IT_&_Software_Development,Software_=_Web_Development ~ Embedded,>=3
1,11176,indiana,INTJR0181976,d18f4cda-19fc-4b2b-8eee-2d4735d9b52e,ALL,Positive Offer+ With Experience,1,3.0,oneAPI Development Software Architect (Remote),bachelors,Experienced,2.0,junior,IT_&_Software_Development,Software_=_Web_Development,<3
2,14367,indiana,INTJR0196855,3745f304-e446-4104-b25b-b4969b539f8b,ALL,Random from other reqs,1,0.0,Analog Engineer,masters,Entry Level,1.0,junior,Engineering,Electrical_=_Electronic_Engineering ~ Hardware...,<3
3,9075,indiana,INTJR0117739,8aabcc5a-f9b5-4a4a-9748-ae273e1744ca,ALL,Positive Offer+ With Experience,0,NaN,Quantum Device Measurement Engineer,doctorate,Experienced,4.0,junior,Engineering,Electrical_=_Electronic_Engineering,>=3
4,9665,indiana,INTJR0185482,9f3ad4ee-f383-45c1-9002-db9387bc8fc8,ALL,Random from other reqs,0,NaN,Graphics Software Engineer,masters,Experienced,3.0,junior,IT_&_Software_Development,Software_=_Web_Development ~ Embedded,>=3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,25861,moscow,MCY823051,9c151fcd-307f-4ec5-85cb-75f9705afa01,ALL,Positive Offer+ With Experience,1,0.0,Pharmacy Technician II,none_required,None,5.0,senior,Medical,Public_Health_Administration,>=3
276,27460,moscow,MCY886723,b16d2372-8163-4690-8fb3-1e3770d09945,ALL,Positive Offer+ With Experience,1,0.0,Lead Security Engineer,none_required,None,8.0,senior,Security,Corporate_Security_Officer,>=3
277,16183,moscow,MCY809698,63f9ce85-2181-459f-8d61-4cba67daebc2,ALL,Positive Offer+ With Experience,0,NaN,Multimedia Producer,bachelors,None,8.0,senior,Design_&_Creative,Industrial_Design,>=3
278,15585,moscow,MCY825060,ea3ed235-6fd4-46ad-80e7-17465179fee9,ALL,Negative Rejected,0,NaN,Senior Consultant-Communications Change Manage...,bachelors,None,5.0,senior,Marketing,Investors_and_Public_=_Media_Relations,>=3
